In [1]:
import pandas as pd
import numpy as np
import uuid
import random
import os

from datetime import datetime, timedelta


In [4]:
def generar_costos_campañas(
    k=10,
    mu_base=1000, sigma_base=200,
    scale_dia=50,
    shape_interaccion=2.0, scale_interaccion=30.0,
    random_state=None
):
    """
    Genera un DataFrame con costos simulados para campañas.

    Parámetros:
    - k: número de campañas
    - mu_base, sigma_base: media y desviación estándar del costo base (Normal)
    - scale_dia: parámetro de escala para el costo por día activo (Exponencial)
    - shape_interaccion, scale_interaccion: parámetros de la distribución Gamma para el costo por interacción
    - random_state: semilla opcional para reproducibilidad

    Retorna:
    - DataFrame con columnas: id_campaña, costo_base_campaña, costo_por_dia_activo_campaña, costo_por_interaccion_campaña
    """
    rng = np.random.default_rng(random_state)

    data = {
        'id_campaña': np.arange(1, k + 1),
        'costo_base_campaña': np.abs(rng.normal(mu_base, sigma_base, size=k)),
        'costo_por_dia_activo_campaña': rng.exponential(scale=scale_dia, size=k),
        'costo_por_interaccion_campaña': rng.gamma(shape=shape_interaccion, scale=scale_interaccion, size=k),
    }

    return pd.DataFrame(data)

In [63]:


def generar_fechas_aleatorias(start_date, end_date, n_dates=1):
    """
    Genera una lista de n_dates fechas aleatorias (con hora) entre start_date y end_date.

    Parámetros:
    - start_date: datetime o string en formato ISO
    - end_date: datetime o string en formato ISO
    - n_dates: número de fechas a generar

    Retorna:
    - Lista de datetime con marca de tiempo (fecha + hora)
    """
    if isinstance(start_date, str):
        start_date = datetime.fromisoformat(start_date)
    if isinstance(end_date, str):
        end_date = datetime.fromisoformat(end_date)

    delta_seconds = int((end_date - start_date).total_seconds())
    
    if n_dates > delta_seconds:
        raise ValueError("Demasiadas fechas para el rango proporcionado.")

    segundos_aleatorios = random.sample(range(delta_seconds), n_dates)
    fechas = [start_date + timedelta(seconds=s) for s in sorted(segundos_aleatorios)]
    
    return fechas

def generate_purchase():
    product_prices =[20,49,99,299,499]
    n_products = random.randint(1,len(product_prices))
    items =0
    purchase =0
    for _ in range(n_products):
        idx = random.randint(1,len(product_prices))-1
        current_item= product_prices[idx]
        current_items_amount =random.randint(1,5)
        current_purchase = current_items_amount*current_item
        items += current_items_amount
        purchase += current_purchase
    return [items,purchase]


In [2]:
# params
n_users=50000
n_visits_mean= 7
n_campaigns = 6
rate_convertions = 0.23
p_campaigns = np.random.choice([2,3,5,7,11,13],n_campaigns)
p_campaigns=p_campaigns/p_campaigns.sum()
p_campaigns_convertion = np.random.choice([3,5,7],n_campaigns)
p_campaigns_convertion=p_campaigns/p_campaigns.sum()
end_date= datetime.now().strftime("%Y-%m-%d %H:%M:%S")
start_date = datetime.now() - timedelta(days=40)


In [62]:
visits_df=os.path.join('..','datasets','visits.csv')
(
    pd.DataFrame( [i for i in range(n_users)] , columns=['user_id'])
    .assign(
        n_visits = lambda df: df.user_id.apply(lambda x: (np.random.poisson(lam=n_visits_mean,size=1)+1)[0]),
        uuid_user = lambda df: df.user_id.apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_DNS,str(f"user-{x}")))),
        ts_visits = lambda df: df.n_visits.apply(lambda x: generar_fechas_aleatorias(start_date=start_date,end_date=end_date,n_dates=x))
    )
    .drop(
        columns=['user_id','n_visits']
    )
    .explode('ts_visits')
        .assign(
        campaign = lambda df: df.uuid_user.apply(lambda x: np.random.multinomial(1,p_campaigns).argmax()+1),
        uuid_interaction = lambda df: df.apply(lambda row: str(uuid.uuid5(uuid.NAMESPACE_DNS,str(f"user-{row['uuid_user']}-{row['campaign']}-{row['ts_visits']}"))),axis=1),
        convertion = lambda df: df.campaign.apply(lambda x: np.random.binomial(1,p_campaigns_convertion[x-1]))
    )
    .sort_values('convertion')
    .to_csv(visits_df,index=False)
    
)

In [66]:
orders_df=os.path.join('..','datasets','orders.csv')


(pd.read_csv(visits_df)
    .query("convertion == 1")
    .assign(
        end_date = lambda df: pd.to_datetime(df.ts_visits, errors='coerce') + timedelta(days=10),
        transaction_ts = lambda df: df.apply(lambda row: generar_fechas_aleatorias(row['ts_visits'],row['end_date'],1),axis=1)
    )
    .explode("transaction_ts")[['uuid_interaction','transaction_ts']]
    .assign(
        transaction = lambda df: df.apply(lambda x: generate_purchase(),axis=1),
        items = lambda df: df.transaction.apply(lambda x: x[0]),
        amount = lambda df: df.transaction.apply(lambda x: x[1]),
    )
    .drop(columns=['transaction'])
    .to_csv(orders_df,index=False)
    
)

In [5]:
campaigns_df=os.path.join('..','datasets','campaigns_details.csv')
generar_costos_campañas(k=n_campaigns,mu_base=90000,sigma_base=100,scale_dia=500,shape_interaccion=11,scale_interaccion=8).to_csv(campaigns_df,index=False)